# Compilation of test data for performance tests

This notebook records the parameters for Wright-Fisher simulations used to generate our test data sets, as well as commands for running covariance estimation algorithms on the test data and compiling the results. 

## Import required libraries and define global variables

In [15]:
# Full library list and version numbers

print('This notebook was prepared using:')

import sys
print('python version %s' % sys.version)

import numpy as np
print('numpy version %s' % np.__version__)

import seaborn as sns
print('seaborn version %s' % sns.__version__)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
print('matplotlib version %s' % mpl.__version__)

import scipy
from scipy import stats
print('scipy version %s' % scipy.__version__)

# GLOBAL VARIABLES

N = 1000        # number of sequences in the population
L = 50          # sequence length
T = 700         # number of generations
MU = 1e-3       # mutation rate
num_selections = 10  # number of sets of selection coefficients that we used
num_trials = 20  # for each set of selection coefficients, we simulated for 20 times
SAMPLE = [1000, 500, 100, 50, 10]  # sampling depth options when subsampling
RECORD = [1, 3, 5, 10]  # sampling time interval options when subsampling
LINEAR = [1, 5, 10, 20, 50, 100, 300]  # list of linear shrinkage strengths/sampling time intervals, e.g., interval=5, linear=5 ==> strength=25'
GAMMA = [5e-5, 5e-4, 5e-3, 5e-2, 5e-1, 1]  # non-linear shrinkage parameter choices that we tested
TRUNCATE = [200, 300, 400, 500, 600, 700]  # truncate options to test performance using different lengths of data
WINDOW = [0, 1, 2, 3, 4, 5, 10, 20, 40, 80, 160]  # window choices that we tested
# loss functions for non-linear shrinkage that we tested
LOSS = ['Fro | $\hat{C}-C$', 'Fro | $\hat{C}^{-1}-C^{-1}$', 'Fro | $C^{-1}\hat{C}-I$', 
        'Fro | $\hat{C}^{-1}C-I$', 'Fro | $\hat{C}^{-1/2}C\hat{C}^{-1/2}-I$', 
        'Nuc | $\hat{C}-C$', 'Nuc | $\hat{C}^{-1}-C^{-1}$', 'Nuc | $C^{-1}\hat{C}-I$', 
        'Nuc | $\hat{C}^{-1}C-I$', 'Nuc | $\hat{C}^{-1/2}C\hat{C}^{-1/2}-I$'] 

# GitHub directories
DATA_DIR = './data'
SELECTION_DIR = './src/selection'
INITIAL_DIR = './src/initial'
SIMULATION_OUTPUT_DIR = './data/simulation_output'
SUBSAMPLE_OUTPUT_DIR = './data/subsample_output'
ESTIMATION_OUTPUT_DIR = './data/estimation_output'
JOB_DIR = './src/jobs'

# relative directories looking from shell scripts in JOB_DIR
SRC_DIR_REL = '..'
DATA_DIR_REL = '../../data'
SELECTION_DIR_REL = '../../src/selection'
INITIAL_DIR_REL = '../../src/initial'
SIMULATION_OUTPUT_DIR_REL = '../../data/simulation_output'
SUBSAMPLE_OUTPUT_DIR_REL = '../../data/subsample_output'
ESTIMATION_OUTPUT_DIR_REL = '../../data/estimation_output'

This notebook was prepared using:
python version 3.6.10 |Anaconda, Inc.| (default, Mar 25 2020, 18:53:43) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
numpy version 1.19.1
seaborn version 0.11.0
matplotlib version 3.3.2
scipy version 1.5.2


## Section 1. Generation of test data through Wright-Fisher simulations¶

Wright-Fisher simulations are performed using src/Wright-Fisher.py. The output of these simulations is saved for processing. The code below creates multiple job files for running many simulations in parallel on a computer cluster.

In [7]:
def generate_selection_coefficients():
    """Generates 10 sets of selection coefficients.
    
    This was how the 10 sets of selection coefficients were generated. 
    Note that since some selectons are randomly sampled, re-run this will yield different selections.
    """
    s = np.zeros((num_selections, L, L), dtype = float)
    sign = np.zeros(2, dtype = int)
    sign[0] = 1
    sign[1] = -1
    for n in range(4):
        num_nonneutral = 10 * (n + 1)
        for i in range(num_nonneutral):
            s[n, i] = 0.05 - 0.1 * i / num_nonneutral

    for n in range(4, num_selections):
        num_nonneutral = 30
        for i in range(num_nonneutral):
            s[n, i] = np.random.choice(sign) * np.random.normal(0.03, 0.01)
            
    for n in range(num_selections):
        np.save(SELECTION_DIR + f'/selection_newly_generated_{n}.npy', s[n])

        
def generate_initial_distribution():
    """Generates the initial distribution of genotypes used in our simulations.
    
    This was how the initial distribution of genotypes were generated. 
    Initially the population mainly consists of three genotypes, each starting with 333 sequences. 
    For every 7 alleles, 1 allele is mutant in all three genotypes. 3 allele are mutant in two genotypes; 
    The other 3 are mutant in only one of three genotypes. 
    """ 
    sequences = []
    counts = []
    seq = np.zeros((3, L), dtype = int)
    for n in range(7):
        seq[0, 7 * n] = seq[1, 7 * n] = seq[2, 7 * n] = 1
        seq[0, 7 * n + 1] = seq[1, 7 * n + 1] = 1
        seq[1, 7 * n + 2] = seq[2, 7 * n + 2] = 1
        seq[0, 7 * n + 3] = seq[2, 7 * n + 3] = 1
        seq[0, 7 * n + 4] = 1
        seq[1, 7 * n + 5] = 1
        seq[2, 7 * n + 6] = 1
    
    sequences.append(np.array([int(i) for i in seq[0]]))
    counts.append(N//3)
    sequences.append(np.array([int(i) for i in seq[1]]))
    counts.append(N//3)
    sequences.append(np.array([int(i) for i in seq[2]]))
    counts.append(N//3)
    sequences.append(np.array([int(i) for i in CM.bin(0, L)]))
    counts.append(N - int(N//3) * 3)
    
    np.savez_compressed(INITIAL_DIR + '/initial', sequences = sequences, counts = counts)
    
    
def print_initial_distribution(init_dist):
    """Prints initial genotypes and their counts."""
    
    for i, seq in enumerate(init_dist['sequences']):
        print(f'Genotype {i+1}', ''.join([str(allele) for allele in seq]), f'count={init_dist["counts"][i]}')
       
    
def generate_shell_script(path2job, jobname, command, hours=4, mem=16, ncpus=1, env='env_pySCA'):
    """Generates a shell script to run on HPCC cluster."""
    
    fp = open(path2job + '/' + jobname + '.sh', 'w')
    fp.write('#!/bin/bash -l\n')
    fp.write(f'#SBATCH --nodes=1\n#SBATCH --ntasks=1\n')
    fp.write(f'#SBATCH --cpus-per-task={ncpus}\n')
    fp.write(f'#SBATCH --mem={mem}G\n#SBATCH --time={hours}:00:00\n')
    fp.write(f'#SBATCH --output={jobname}.stdout\n')
    fp.write(f'#SBATCH --job-name="{jobname}"\n')
    fp.write(f'date\nsource activate {env}\n')
    fp.write(command)
    fp.write('\ndate')
    fp.close()
    
    
def load_selection(s):
    """Loads one of 10 sets of selection coefficients."""
    
    return np.load(SELECTION_DIR + f'/selection_{s}.npy')

In [8]:
# 10 sets of selection coefficients used for simulations
selections = np.zeros((num_selections, L), dtype=float)
for s in range(num_selections):
    selections[s] = load_selection(s)
        
# initial distribution of genotypes in the population
initial = np.load(INITIAL_DIR + f'/initial.npz')
print_initial_distribution(initial)

Genotype 1 11011001101100110110011011001101100110110011011000 count=333
Genotype 2 11100101110010111001011100101110010111001011100100 count=333
Genotype 3 10110011011001101100110110011011001101100110110010 count=333
Genotype 4 00000000000000000000000000000000000000000000000000 count=1


In [9]:
# In total, num_selections x num_trials = 200 simulations
for s in range(num_selections):
    for n in range(num_trials):
        jobname = f'simulation_s={s}_n={n}'
        job_pars = {'-N': N,
                    '-L': L,
                    '-T': T,
                    '-i': f'{INITIAL_DIR_REL}/initial.npz',
                    '-s': f'{SELECTION_DIR_REL}/selection_{s}.npy',
                    '-o': f'{SIMULATION_OUTPUT_DIR_REL}/simulation_output_s={s}_n={n}',
                    '--mu': MU}
        command = 'python ../Wright-Fisher.py '
        command += ' '.join([k + ' ' + str(v) for k, v in job_pars.items()])
        command += '\n'
        generate_shell_script(JOB_DIR, jobname, command, hours=1)
    
jobname = 'simulation_submission'
command = ''
for s in range(num_selections):
    for n in range(num_trials):
        command += f'sbatch simulation_s={s}_n={n}.sh\n'
generate_shell_script(JOB_DIR, jobname, command)

## Section 2. Subsample the simulated results for later inference under various limited sampling scenarios
We subsample the simulated results varying sampling depth and time interval. The output of is saved for investigating performance of inference under different limited sampling effects. The code below creates multiple files for running many subsampling jobs in parallel on a computer cluster.

In [10]:
# In total, num_selections x num_trials x len(SAMPLE) x len(RECORD) = 4000 subsampling runs, put as 200 jobs
for s in range(num_selections):
    for n in range(num_trials):
        jobname = f'subsample_s={s}_n={n}'
        command = ''
        for sample in SAMPLE:
            for record in RECORD:
                job_pars = {'-i': f'{SIMULATION_OUTPUT_DIR_REL}/simulation_output_s={s}_n={n}.npz', 
                            '-o': f'{SUBSAMPLE_OUTPUT_DIR_REL}/subsample_output_s={s}_n={n}_sample={sample}_record={record}', 
                            '--sample': sample, 
                            '--record': record, 
                            '--compact': '',
                            '--intCovAtTimes': ''}
                command += 'python ../subsample.py '
                command += ' '.join([k + ' ' + str(v) for k, v in job_pars.items()])
                command += '\n'
        generate_shell_script(JOB_DIR, jobname, command, hours=1)
jobname = 'subsample_submission'
command = ''
for s in range(num_selections):
    for n in range(num_trials):
        command += f'sbatch subsample_s={s}_n={n}.sh\n'
generate_shell_script(JOB_DIR, jobname, command)

## Section 3. Running the covariance estimation algorithms and compiling output
Finally we estimate covariance and infer selection coefficients for all subsampled cases generated above. The output of is saved for plotting. The code below creates multiple files to run in parallel on a computer cluster.

In [16]:
# In total, len(TRUNCATE) x len(WINDOW) = 42 jobs
for tr in TRUNCATE:
    for win in WINDOW:
        jobname = f'estimate_truncate={tr}_window={win}'
        command = ''
        job_pars = {'-N': N,
                    '-L': L,
                    '-T': T,
                    '-truncate': tr,
                    '-window': win,
                    '-num_selections': num_selections,
                    '-num_trials': num_trials,
                    '-s': f'{SELECTION_DIR_REL}/selection',
                    '-i': f'{SUBSAMPLE_OUTPUT_DIR_REL}/subsample_output', 
                    '-o': f'{ESTIMATION_OUTPUT_DIR_REL}/estimation_output', 
                    '--minimal_size': ''}
        command += 'python ../estimate.py '
        command += ' '.join([k + ' ' + str(v) for k, v in job_pars.items()])
        command += '\n'
        generate_shell_script(JOB_DIR, jobname, command, hours=8)
jobname = 'estimate_submission'
command = ''
for tr in TRUNCATE:
    for win in WINDOW:
        command += f'sbatch estimate_truncate={tr}_window={win}.sh\n'
generate_shell_script(JOB_DIR, jobname, command)